In [1]:
import tkinter as tk
from tkinter import ttk, messagebox, filedialog
import cv2 as cv
import os
import numpy as np
from mtcnn.mtcnn import MTCNN
from keras_facenet import FaceNet
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pickle
import uuid
import time
import shutil
import csv

class StudentDataApp(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("Student Data Entry & Recognition")
        self.geometry("800x600")

        # Initialize face recognition components
        self.detector = MTCNN()
        self.embedder = FaceNet()
        self.encoder = LabelEncoder()
        self.model = None  # Placeholder for SVM model

        # Initialize image paths
        self.image_paths = []

        # Load existing face embeddings and labels
        self.load_existing_data()

        # Create Tab Control
        self.tab_control = ttk.Notebook(self)
        
        # Create Tabs
        self.data_entry_tab = ttk.Frame(self.tab_control)
        self.attendance_tab = ttk.Frame(self.tab_control)
        
        # Add Tabs to Tab Control
        self.tab_control.add(self.data_entry_tab, text='Data Entry')
        self.tab_control.add(self.attendance_tab, text='Attendance')
        
        # Pack Tab Control
        self.tab_control.pack(expand=1, fill='both')

        # Initialize UI for Data Entry Tab
        self.init_data_entry_ui()

        # Initialize UI for Attendance Tab
        self.init_attendance_ui()

    def init_data_entry_ui(self):
        # ID Label and Entry
        self.label_id = ttk.Label(self.data_entry_tab, text="Student ID:")
        self.label_id.pack(pady=5)
        self.entry_id = ttk.Entry(self.data_entry_tab)
        self.entry_id.pack(pady=5)

        # Name Label and Entry
        self.label_name = ttk.Label(self.data_entry_tab, text="Student Name:")
        self.label_name.pack(pady=5)
        self.entry_name = ttk.Entry(self.data_entry_tab)
        self.entry_name.pack(pady=5)

        # Image Listbox
        self.label_image = ttk.Label(self.data_entry_tab, text="Selected Images:")
        self.label_image.pack(pady=5)
        self.listbox_images = tk.Listbox(self.data_entry_tab, height=8)
        self.listbox_images.pack(pady=5)

        # Upload Button
        self.upload_button = ttk.Button(self.data_entry_tab, text="Upload Images", command=self.upload_images)
        self.upload_button.pack(pady=5)

        # Capture Button
        self.capture_button = ttk.Button(self.data_entry_tab, text="Capture Images", command=self.capture_images)
        self.capture_button.pack(pady=5)

        # Save Button
        self.save_button = ttk.Button(self.data_entry_tab, text="Save", command=self.save_data)
        self.save_button.pack(pady=20)

        # Train Button
        self.train_button = ttk.Button(self.data_entry_tab, text="Train New Student", command=self.train_new_student, state=tk.DISABLED)
        self.train_button.pack(pady=10)

        # Face Recognition Button
        self.face_recognition_button = ttk.Button(self.data_entry_tab, text="Face Recognition", command=self.face_recognition)
        self.face_recognition_button.pack(pady=5)

    def init_attendance_ui(self):
        # Course Label and Entry
        self.label_course = ttk.Label(self.attendance_tab, text="Course:")
        self.label_course.pack(pady=5)
        self.entry_course = ttk.Entry(self.attendance_tab)
        self.entry_course.pack(pady=5)

        # Date Label and Entry
        self.label_date = ttk.Label(self.attendance_tab, text="Date (YYYY-MM-DD):")
        self.label_date.pack(pady=5)
        self.entry_date = ttk.Entry(self.attendance_tab)
        self.entry_date.pack(pady=5)

        # Attendance Button
        self.attendance_button = ttk.Button(self.attendance_tab, text="Take Attendance", command=self.take_attendance)
        self.attendance_button.pack(pady=10)

        # Attendance Table
        self.attendance_table = ttk.Treeview(self.attendance_tab, columns=('ID', 'Name', 'Date', 'Time'), show='headings')
        self.attendance_table.heading('ID', text='ID')
        self.attendance_table.heading('Name', text='Name')
        self.attendance_table.heading('Date', text='Date')
        self.attendance_table.heading('Time', text='Time')
        self.attendance_table.pack(padx=10, pady=10)

    def load_existing_data(self):
        # Load existing face embeddings and labels if available
        if os.path.exists('faces_embeddings_done_4classes.npz'):
            data = np.load('faces_embeddings_done_4classes.npz')
            self.X = data['arr_0']
            self.Y = data['arr_1']
            self.encoder.fit(self.Y)  # Fit label encoder
        else:
            self.X = np.empty((0, 160, 160, 3))
            self.Y = np.array([])

    def upload_images(self):
        file_paths = filedialog.askopenfilenames(
            title="Select Images",
            filetypes=[("Image files", "*.jpg *.jpeg *.png")]
        )
        if file_paths:
            self.image_paths.extend(file_paths)
            for path in file_paths:
                # Copy the uploaded image to dataset/id folder
                student_id = self.entry_id.get()
                if not student_id:
                    messagebox.showwarning("Input Error", "Please enter the Student ID before uploading images.")
                    return
                
                IMAGES_PATH = os.path.join("dataset", student_id)
                os.makedirs(IMAGES_PATH, exist_ok=True)
                
                dest_path = os.path.join(IMAGES_PATH, os.path.basename(path))
                try:
                    shutil.copyfile(path, dest_path)
                    self.listbox_images.insert(tk.END, os.path.basename(dest_path))
                except Exception as e:
                    messagebox.showerror("Error", f"Error copying file: {str(e)}")

    def capture_images(self):
        student_id = self.entry_id.get()
        student_name = self.entry_name.get()
        
        if not student_id or not student_name:
            messagebox.showwarning("Input Error", "Please enter both Student ID and Name before capturing images.")
            return
        
        IMAGES_PATH = os.path.join("dataset", student_id)
        os.makedirs(IMAGES_PATH, exist_ok=True)
        
        number_images = 60
        cap = cv.VideoCapture(0)

        if not cap.isOpened():
            messagebox.showerror("Error", "Cannot open camera.")
            return

        captured_image_paths = []

        for imgnum in range(number_images):
            ret, frame = cap.read()
            if not ret:
                messagebox.showerror("Error", "Failed to capture image.")
                break

            imgname = os.path.join(IMAGES_PATH, f'{str(uuid.uuid1())}.jpg')
            cv.imwrite(imgname, frame)
            captured_image_paths.append(imgname)
            
            cv.imshow('frame', frame)
            if cv.waitKey(1) & 0xFF == ord('q'):
                break
            time.sleep(0.5)

        cap.release()
        cv.destroyAllWindows()

        self.image_paths.extend(captured_image_paths)
        for path in captured_image_paths:
            self.listbox_images.insert(tk.END, os.path.basename(path))

    def save_data(self):
        student_id = self.entry_id.get()
        student_name = self.entry_name.get()
        
        if not student_id or not student_name or not self.image_paths:
            messagebox.showwarning("Input Error", "Please enter ID, Name, and select images.")
            return
        
        try:
            # Save student data to CSV
            csv_file = 'student_data.csv'
            if not os.path.exists(csv_file):
                with open(csv_file, mode='w', newline='') as file:
                    writer = csv.writer(file)
                    writer.writerow(['ID', 'Name', 'ImagePath'])

            # Write the student information
            with open(csv_file, mode='a', newline='') as file:
                writer = csv.writer(file)
                writer.writerow([student_id, student_name, self.image_paths[0]])  # Save the first image path

            # Update UI state
            self.train_button.config(state=tk.NORMAL)  # Enable train button after saving
            
            # Clear the input fields and image listbox
            self.entry_id.delete(0, tk.END)
            self.entry_name.delete(0, tk.END)
            self.listbox_images.delete(0, tk.END)
            self.image_paths = []
            
            messagebox.showinfo("Success", "Student data saved successfully.")
        
        except Exception as e:
            messagebox.showerror("Error", f"Error saving data: {str(e)}")

    def train_new_student(self):
        class FACELOADING:
            def __init__(self, directory):
                self.directory = directory
                self.detector = MTCNN()
                self.target_size = (160, 160)
                self.X = []
                self.Y = []

            def extract_face(self, filename):
                try:
                    img = cv.imread(filename)
                    img_rgb = cv.cvtColor(img, cv.COLOR_BGR2RGB)
                    results = self.detector.detect_faces(img_rgb)
                    if results:
                        x1, y1, width, height = results[0]['box']
                        x2, y2 = x1 + width, y1 + height
                        face = img_rgb[y1:y2, x1:x2]
                        image = cv.resize(face, self.target_size)
                        return image
                except Exception as e:
                    print(f"Error extracting face: {str(e)}")
                return None
            
            def load_faces(self, subdir):
                faces = []
                y = subdir
                subdir_full = os.path.join(self.directory, subdir)
                for filename in os.listdir(subdir_full):
                    path = os.path.join(subdir_full, filename)
                    face = self.extract_face(path)
                    if face is not None:
                        faces.append(face)
                return faces, y
            
            def load_dataset(self):
                for subdir in os.listdir(self.directory):
                    subdir_full = os.path.join(self.directory, subdir)
                    if not os.path.isdir(subdir_full):
                        continue
                    faces, y = self.load_faces(subdir)
                    labels = [y] * len(faces)
                    print(f'Loaded {len(faces)} examples for class: {y}')
                    self.X.extend(faces)
                    self.Y.extend(labels)
                return np.array(self.X), np.array(self.Y)
        
        directory = "dataset"
        faceloading = FACELOADING(directory)
        self.X, self.Y = faceloading.load_dataset()

        self.encoder.fit(self.Y)
        Y = self.encoder.transform(self.Y)
        self.model = SVC(kernel='linear', probability=True)

        X = np.array([self.embedder.embeddings(face) for face in self.X])
        X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
        
        self.model.fit(X_train, Y_train)
        Y_pred = self.model.predict(X_test)
        accuracy = accuracy_score(Y_test, Y_pred)
        
        messagebox.showinfo("Training Complete", f"Model trained with accuracy: {accuracy * 100:.2f}%")

        with open('svm_model_160x160.pkl', 'wb') as file:
            pickle.dump(self.model, file)

        np.savez_compressed('faces_embeddings_done_4classes.npz', self.X, self.Y)

    def face_recognition(self):
        cap = cv.VideoCapture(0)
        if not cap.isOpened():
            messagebox.showerror("Error", "Cannot open camera.")
            return

        with open('svm_model_160x160.pkl', 'rb') as file:
            self.model = pickle.load(file)

        while True:
            ret, frame = cap.read()
            if not ret:
                messagebox.showerror("Error", "Failed to capture image.")
                break
            
            frame_rgb = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
            faces = self.detector.detect_faces(frame_rgb)
            for result in faces:
                x1, y1, width, height = result['box']
                x2, y2 = x1 + width, y1 + height
                face = frame_rgb[y1:y2, x1:x2]
                face = cv.resize(face, (160, 160))
                face_emb = self.embedder.embeddings(face[np.newaxis, :])
                face_emb = face_emb / np.linalg.norm(face_emb, axis=1, keepdims=True)
                predictions = self.model.predict(face_emb)
                pred_name = self.encoder.inverse_transform(predictions)[0]

                # Draw bounding box and label on the image
                cv.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv.putText(frame, pred_name, (x1, y1 - 10), cv.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
            
            cv.imshow('Face Recognition', frame)
            if cv.waitKey(1) & 0xFF == ord('q'):
                break
        
        cap.release()
        cv.destroyAllWindows()

    def take_attendance(self):
        course = self.entry_course.get()
        date = self.entry_date.get()
    
        if not course or not date:
            messagebox.showwarning("Input Error", "Please enter both Course and Date.")
            return
    
        # Create the course folder if it doesn't exist
        course_folder = os.path.join('attendance', course)
        os.makedirs(course_folder, exist_ok=True)
    
        # File path for the attendance CSV
        file_path = os.path.join(course_folder, f"{date}.csv")
    
        # Load student data from CSV
        student_data = {}
        with open('student_data.csv', mode='r') as file:
            reader = csv.DictReader(file)
            for row in reader:
                student_data[row['ID']] = row['Name']
    
        cap = cv.VideoCapture(0)
        if not cap.isOpened():
            messagebox.showerror("Error", "Cannot open camera.")
            return
    
        with open('svm_model_160x160.pkl', 'rb') as file:
            self.model = pickle.load(file)
    
        attendance_records = []
        recorded_students = set()  # To keep track of recorded student IDs
    
        while True:
            ret, frame = cap.read()
            if not ret:
                messagebox.showerror("Error", "Failed to capture image.")
                break
            
            frame_rgb = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
            faces = self.detector.detect_faces(frame_rgb)
            for result in faces:
                x1, y1, width, height = result['box']
                x2, y2 = x1 + width, y1 + height
                face = frame_rgb[y1:y2, x1:x2]
                face = cv.resize(face, (160, 160))
                face_emb = self.embedder.embeddings(face[np.newaxis, :])
                face_emb = face_emb / np.linalg.norm(face_emb, axis=1, keepdims=True)
                predictions = self.model.predict(face_emb)
                pred_id = self.encoder.inverse_transform(predictions)[0]
    
                # Use student ID to get the student name
                pred_name = student_data.get(pred_id, "Unknown")
    
                # Draw bounding box and label on the image
                cv.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv.putText(frame, pred_name, (x1, y1 - 10), cv.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
    
                # Check if the student has already been recorded
                if pred_id not in recorded_students:
                    # Record attendance
                    current_time = time.strftime("%H:%M:%S")
                    attendance_records.append([pred_id, pred_name, date, current_time])
    
                    # Add the student ID to the set
                    recorded_students.add(pred_id)
    
            cv.imshow('Face Recognition', frame)
            if cv.waitKey(1) & 0xFF == ord('q'):
                break
        
        cap.release()
        cv.destroyAllWindows()
    
        # Save attendance records to CSV
        with open(file_path, mode='w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['ID', 'Name', 'Date', 'Time'])
            writer.writerows(attendance_records)
        
        # Update attendance table
        self.update_attendance_table(file_path)
    
    def update_attendance_table(self, attendance_file):
        for row in self.attendance_table.get_children():
            self.attendance_table.delete(row)
    
        with open(attendance_file, mode='r') as file:
            reader = csv.reader(file)
            next(reader)  # Skip header row
            for row in reader:
                self.attendance_table.insert("", tk.END, values=row)

# Run the application
if __name__ == "__main__":
    app = StudentDataApp()
    app.mainloop()


1/1 [==============================] - 0s 16ms/step
